In [1]:
# !pip install netCDF4
# !pip install sympy==1.7.1


In [2]:
import datetime as dt  # Python standard library datetime  module
import numpy as np
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/


def ncdump(path , verb=True):
    
    nc_fid = Dataset(path)

    
    '''
    ncdump outputs dimensions, variables and their attribute information.
    The information is similar to that of NCAR's ncdump utility.
    ncdump requires a valid instance of Dataset.

    Parameters
    ----------
    nc_fid : netCDF4.Dataset
        A netCDF4 dateset object
    verb : Boolean
        whether or not nc_attrs, nc_dims, and nc_vars are printed

    Returns
    -------
    nc_attrs : list
        A Python list of the NetCDF file global attributes
    nc_dims : list
        A Python list of the NetCDF file dimensions
    nc_vars : list
        A Python list of the NetCDF file variables
    '''
    def print_ncattr(key):
        """
        Prints the NetCDF file attributes for a given key

        Parameters
        ----------
        key : unicode
            a valid netCDF4.Dataset.variables key
        """
        try:
            print("\t\ttype:", repr(nc_fid.variables[key].dtype))
            for ncattr in nc_fid.variables[key].ncattrs():
                print('\t\t%s:' % ncattr)
                repr(nc_fid.variables[key].getncattr(ncattr))
        except KeyError:
            print("\t\tWARNING: %s does not contain variable attributes" % key)

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    if verb:
        print("NetCDF Global Attributes:")
        for nc_attr in nc_attrs:
            print('\t%s:' % nc_attr, repr(nc_fid.getncattr(nc_attr)))
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print("NetCDF dimension information:")
        for dim in nc_dims:
            print("\tName:", dim )
            print("\t\tsize:", len(nc_fid.dimensions[dim]))
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    if verb:
        print("NetCDF variable information:")
        for var in nc_vars:
            if var not in nc_dims:
                print('\tName:', var)
                print("\t\tdimensions:", nc_fid.variables[var].dimensions)
                print("\t\tsize:", nc_fid.variables[var].size)
                print_ncattr(var)
    return(nc_attrs, nc_dims, nc_vars)




#     dataset = Dataset('../data/TIEGCM_CCMC/my_stuff/s_002.nc')
#     #dataset = Dataset('../data/TIEGCM_CCMC/s010.nc')

#     ncdump(dataset)


#     data = Dataset('../data/TIEGCM_CCMC/my_stuff/s_002.nc')
#     lon = np.array(data.variables['lon'])
#     lat = np.array(data.variables['lat'])
#     print lon
#     print lat

In [9]:
file_dir = '/data/data_geodyn/atmos_models_data/tiegcm/2003/' 

file = file_dir+'s320.nc'

# ncdump(file , verb=True)

In [11]:
cdf_data = Dataset(file, 'r')


In [21]:
cdf_data['ilev'][:]

masked_array(data=[-7. , -6.5, -6. , -5.5, -5. , -4.5, -4. , -3.5, -3. ,
                   -2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,
                    2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  5.5,  6. ,
                    6.5,  7. ],
             mask=False,
       fill_value=1e+20)

In [7]:
# from os import path
# import time as ti
# import glob
# import numpy as np
# from netCDF4 import Dataset
# from datetime import datetime, timezone
# from kamodo import Kamodo



# """
# Created on Mon June 14, 2021.
# @author: Zach Waldron
# """

# # from os import path
# # import numpy as np
# # import time as ti
# # from datetime import datetime, timezone
# # from netCDF4 import Dataset
# # from kamodo import Kamodo
# # import kamodo.readers.reader_plotutilities as RPlot
# # import kamodo.readers.reader_utilities as RU


# '''
# This code reads the TIEGCM model output (AKA secondary history files).
# Each sechist file consists of 1 day of data for the TIEGCM.
# '''


# ### Make a dict of the possible variable names in TIEGCM
# ###       the intended format is:   "Output VarName":['Latex_Varname', 'Latex_Unit' ]
# tiegcm_varnames={
#                  ### 4D Variables, vertical coordinate on midpoint levels (lev)
#                  "ZGMID"    : ["ZG_{mid}","cm"],    # geometric height- interpolated to the mid points
#                  "TN"       : ["T_N","K"],          # neutral temperature    
#                  "O2"       : ["O_2","mmr"],        # molecular oxygen   
#                  "O1"       : ["O_1","mmr"],        # atomic oxygen    
#                  "N2"       : ["N_2","mmr"],        # molecular nitrogen
#                  "HE"       : ["He","mmr"],         # helium     
#                  "NO"       : ["NO","mmr"],         # nitric oxide  
#                  "N2"       : ["N_{2}","mmr"],      # molecular nitrogen 
#                  "N4S"      : ["N4S","mmr"],        #  N4S ?
#                  "TE"  : ["T_e","K"],         #  ELECTRON TEMPERATURE
#                  "TI"  : ["T_i","K"],         #  ION TEMPERATURE
#                  "O2P" : ["O_2+","1/cm**3"],  #  O2+ ION
#                  "OP"  : ["O+","1/cm**3"],    #   O+ ION
#                  "N2N"      : ["N_{2,N}","mmr"],    # molecular nitrogen (maybe number density)
#                  "CO2_COOL" : ["CO_{2}_{cool}","erg/g/s"],  #  CO2 cooling rates
#                  "NO_COOL"  : ["NO_{COOL}","erg/g/s"],      #  NO cooling rates
#                  "UN"  : ["U_N","cm/s"],            #  neutral ZONAL wind (+EAST)
#                  "VN"  : ["V_N","cm/s"],            #  neutral MERIDIONAL wind (+NORTH)
#                 # 
#                 ### 4D Variables, vertical coordinate on interface levels (ilev)
#                  "DEN"      :["rho","g/cm**3"],     # total neutral mass density  
#                  "ZG"       :["Z_G","cm"],          # geometric height  
#                  "Z"        :["Z","cm"],            # heopotential height (cm)  
#                  "NE"       : ["N_e","1/cm**3"],    #  ELECTRON DENSITY
#                  "OMEGA"    : ["omega","1/s"],      #  VERTICAL MOTION
#                  "POTEN"    : ["poten","V"],        #  ELECTRIC POTENTIAL
#                 ### 4D Variables, vertical coordinate on interface mag levels (imlev)
#                  "ZGMAG"  : ["ZG_{mag}",""],     #  Geopotential Height on Geomagnetic Grid (km)
#                 #
#                 ### 3D Variables,    (time, lat, lon)
#                  "TEC"  : ["TEC","1/cm**2"],     #  Total Electron Content
#                  "TLBC"  : ["TLBC","K"],       #  Lower boundary condition for TN
#                  "ULBC"  : ["ULBC","cm/s"],    #  Lower boundary condition for UN
#                  "VLBC"  : ["VLBC","cm/s"],    #  Lower boundary condition for VN
#                  "TLBC_NM"  : ["TLBC_NM",""],  #  Lower boundary condition for TN (TIME N-1)
#                  "ULBC_NM"  : ["ULBC_NM",""],  #  Lower boundary condition for UN (TIME N-1)
#                  "VLBC_NM"  : ["VLBC_NM",""],  #  Lower boundary condition for VN (TIME N-1)
#                   }

# #####--------------------------------------------------------------------------------------
# ##### Define some helpful functions for dealing with time systems
# @np.vectorize
# def dts_to_hrs(datetime_string, filedate):
#     '''Get hours since midnight from datetime string'''
    
#     return (datetime.strptime(datetime_string, '%Y-%m-%d %H:%M:%S').replace(tzinfo=timezone.utc)\
#             -filedate).total_seconds()/3600.

# @np.vectorize
# def filename_to_dts(filename, string_date):
#     '''Get datetime string in format "YYYY-DDD HH:mm:SS" from filename'''
    
# #     mmhhss = filename.split('/')[-1].split('\\')[-1][12:18]
#     return string_date+' '+mmhhss[:2]+':'+mmhhss[2:4]+':'+mmhhss[4:] 

# def dts_to_ts(file_dts):
#     '''Get datetime timestamp in UTC from datetime string'''    
    
#     return datetime.timestamp(datetime.strptime(file_dts, '%Y-%j %H:%M:%S'
#                                                 ).replace(tzinfo=timezone.utc))

# #####--------------------------------------------------------------------------------------
# ### Construct a TEIGCM class that inherits Kamodo       
# class TIEGCM(Kamodo): 
#     def __init__(self, file_path, filenames, variables_requested=[], runname="noname",
#                  filetimes=False, verbose=False, gridded_int=True, printfiles=True,
#                  **kwargs): 
        
#         #### Use a super init so that your class inherits any methods from Kamodo
#         super(TIEGCM, self).__init__()
        
        
# #         #check if given .nc file exists. If not, convert files with same prefix to netCDF
# #         if not path.isfile(file_prefix+'.nc'):
# #             from kamodo.readers.swmfie_tocdf import convertSWMFIE_toCDF
# #             test = convertSWMFIE_toCDF(file_prefix)  
# #             if not test: 
# #                 self.conversion_test = test  #only needed for 1 file/time cases
# #                 return    #if file conversion fails, return 0
# #         t0 = ti.perf_counter()


        
#         ####   Establish the time attributes that will be
#         ####   used to identify files by their date
# #         year = filename.split('/')[-2]
# #         doy  = filename.split('/')[-1].split('\\')[0][1:-3]
# #         self.filedate = datetime.strptime(string_date+' 00:00:00', \
# #                                           '%Y-%j %H:%M:%S').replace(tzinfo=timezone.utc) #dt object
        
#         #### establish beginning time and end time of file list
#         ####  
# #         cdf_data = Dataset(filename+'.nc', 'r')
#         for i in filenames:
#             print(i)
# #         files = filenames
#         self.datetimes = list(filename_to_dts([files[0], files[-1]], string_date))  #strings in format = YYYY-MM-DD HH:MM:SS 
#         self.timerange0={'min':self.datetimes[0], 'max':self.datetimes[-1], 'n':len(files)}
#         self.timerange = self.timerange0
#         self.filetimes=[dts_to_ts(file_dts) for file_dts in self.datetimes]   #timestamps in UTC     
#         if filetimes: 
#             return           
    
#         ####  add condition that  exits code if there is only 1 file
#         ####   interpolator code will break if only 1 file?
#         if len(files)<2:
#             print('Not enough files found with given file prefix.')
#             return
        
#         #### Store our inputs as class attributes to the class
#         self.filename = files
#         self.runname = runname
#         self.missing_value = np.NAN
#         self._registered = 0
#         self.variables=dict()
#         if printfiles: 
#             print('Files:')
#             for file in self.filename: print(file)

#         #### Access the first file and get a list of the 
#         ####     variable names in this file set
#         #### This is merely a check to see if the requested
#         ####     variables are in the files
#         if len(variables_requested)>0:
#             gvar_list = [key for key in cdf_data.variables.keys() if key \
#                          in variables_requested]
#             if len(gvar_list)!=len(variables_requested):
#                 err_list = [item for item in variables_requested if item not in \
#                             cdf_data.variables.keys()]
#                 print('Some requested variables are not available:', err_list)
#         else:
#             gvar_list = [key for key in cdf_data.variables.keys() \
#                          if key not in cdf_data.dimensions.keys() and key not in \
#                              ['theta_Btilt', 'psi_Btilt']]  
        
#         #### Store coordinate data as class attributes    
#         self._time = np.array(cdf_data.variables['time'])
#         self._lev = np.array(cdf_data.variables['lev'])
#         self._ilev = np.array(cdf_data.variables['ilev'])
#         self._lat = np.array(cdf_data.variables['lat'])
#         self._lon = np.array(cdf_data.variables['lon'])
#         # self._height = np.array(cdf_data.variables['height'])

        
#         #### Store the requested variables into a dictionary 
#         ####     as the variables attributes
#         #### This will contain units, dtype, and the data
#         self.variables = {key:{'units':cdf_data.variables[key].units, 'dtype':np.float32,
#                                'data':np.array(cdf_data.variables[key])}\
#                           for key in gvar_list}
#         #### Close the CDF file
#         cdf_data.close()
#         if verbose: print(f'Took {ti.perf_counter()-t0:.6f}s to read in data')
            
            
#         #### register interpolators for each  requested variable
#         varname_list = [key for key in self.variables.keys()]  #store original list b/c gridded interpolators
#         t_reg = ti.perf_counter()
#         for varname in varname_list:  #all are 3D variables
#             self.register_3D_variable(self.variables[varname]['units'], 
#                                       self.variables[varname]['data'], varname,
#                                       gridded_int)
#         if verbose: print(f'Took {ti.perf_counter()-t_reg:.5f}s to register '+\
#                           f'{len(varname_list)} variables.')
#         self = RPlot.initialize_4D_plot(self)  #initialize 4D?!?!?!?!?!?!?!?!?!?!?!?!?!?!?! plotting variables  
#         if verbose: print(f'Took a total of {ti.perf_counter()-t0:.5f}s to kamodofy '+\
#                           f'{len(gvar_list)} variables.')

#     #### Define and register a 3D variable-----------------------------------------
#     def register_3D_variable(self, units, variable, varname, gridded_int):
#         """Registers a 3d interpolator with 3d signature"""
        
#         #define and register the interpolators
#         xvec_dependencies = {'time':'s','lat':'deg','lon':'deg'}
#         self = RU.regdef_3D_interpolators(self, units, variable, self._time, 
#                                        self._lat, self._lon, varname, 
#                                        xvec_dependencies, gridded_int)       
#         return 
    
    
#     #### Define and register a 4D variable -----------------------------------------
#     def register_4D_variable(self, units, variable, varname, gridded_int):
#         """Registers a 4d interpolator with 4d signature"""

#         #determine which file the variable came from
#         for i in range(len(self.patterns)-1,-1,-1):  #go in reverse to account for overwritten variables
#             if varname in self.varfiles[str(i)]:
#                 lat = getattr(self, '_lat'+str(i))  #get the correct coordinates
#                 lon = getattr(self, '_lon'+str(i))
#                 height = getattr(self, '_height'+str(i))
#                 #print(varname, self.patterns[i])
#                 break
        
#         #define and register the interpolators
#         xvec_dependencies = {'time':'s','lev':'unitless','lat':'deg','lon':'deg'}
#         self = RU.regdef_4D_interpolators(self, units, variable, self._time,
#                                           lev, lat, lon,
#                                           varname, xvec_dependencies, gridded_int)
#         return


# # #begin plotting code -----------------------------------
# #     def set_plot(self, var, plottype, cutV=400., cutL=0, 
# #                  timerange={}, lonrange={}, latrange={}, htrange={}):
# #         '''Set plotting variables for available preset plot types.'''
        
# #         tic = ti.perf_counter()  #start timer
# #         test = RPlot.if_new_plot(self, var, plottype, cutV, cutL, timerange, 
# #                                  lonrange, latrange, htrange)
# #         if test==0: return
# #         else: self=test
# #         self = RU.setup_interpolating_grids(self,var)
# #         toc = ti.perf_counter()  #end timer
# #         print(f"Time resetting plot and precomputing interpolations: {toc - tic:0.4f} seconds")
# #         return
    
# #     def get_plot(self, var, colorscale="Viridis",datascale="linear", ellipse=False):
# #         '''
# #         Return a plotly figure object for the available plot types set in set_plot()..
# #         colorscale = Viridis [default], Cividis, BlueRed or Rainbow
# #         '''
        
# #         self.gridSize=len(self._height)*len(self._lat)*len(self._lon)
# #         print(f'set_plot::colorscale={colorscale}, datascale={datascale}')
# #         print(f'Run: {self.runname}')
# #         #Set some text strings
# #         txtbot=f"Model: SWMF_IE, dt={self.dt:.4f} hrs, "+\
# #             f"dlat={self.dlat:.1f} deg, dlon={self.dlon:.1f} deg, dz={self.dz:.1f} km."
# #         #{self.gridSize} volume cells,
            
# #         #set plot variables
# #         xint, yint, nx, ny, kamodo_plot, xlabel, ylabel, xformat, yformat, \
# #             zformat, xunit, yunit, txttop, txtbar, result = RPlot.set_plotvar(self, datascale, var)
# #         if 'TimeLon' in self.plottype: 
# #             plot_flip = True
# #         elif 'TimeLat' in self.plottype and self.nDim==4:
# #             plot_flip = True
# #         else:
# #             plot_flip = False
            
# #         #get plot with chosen settings
# #         fig = RPlot.heatplot2D(xint, yint, plot_flip, nx, ny, result, datascale, kamodo_plot, 
# #                xlabel, ylabel, colorscale, txtbar, xformat, yformat, zformat, 
# #                xunit, yunit, txttop, txtbot, ellipse=ellipse)

# #         return fig
    
# #     def make_plot(self, var, plottype, cutV=0, cutL=0, timerange={},
# #                  lonrange={}, latrange={}, htrange={}, 
# #                  colorscale="Viridis", datascale="linear", ellipse=False):
# #         '''Simplified call for plots. Execute with iplot(self.make_plot(....))
# #         Possible plottypes: LonLat, LonH, LatH, TimeLon, TimeLat, TimeH'''
        
# #         test = self.set_plot(var, plottype, cutV=cutV, cutL=cutL, timerange=timerange,
# #                  lonrange=lonrange, latrange=latrange, htrange=htrange)
# #         if test==1: return {} #if plottype requested invalid for variable, do nothing
# #         fig = self.get_plot(var, colorscale=colorscale, datascale=datascale, ellipse=ellipse)
# #         return fig        











In [8]:
# !pip list
